In [1]:
from time import time
import sys
sys.path.append('/models/StateFarmChallenge')
import os
import logging

from six.moves import xrange
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from submission import Submission

import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# Create queue for csv file
queue = tf.train.string_input_producer([''], shuffle=True, capacity=32, name='enqueue_op')
# Reader to reada csv rows
reader = tf.TextLineReader()
key, value = reader.read(queue)

